**PROCESADO**

**1. Preparación del entorno**

In [3]:
try:
  from google.colab import auth
  auth.authenticate_user()
  from googleapiclient.discovery import build
  from google.cloud import storage
  from unidecode import unidecode
  from collections import Counter
  from itertools import combinations
  from nltk.corpus import stopwords
  from nltk.tokenize import RegexpTokenizer
  from sklearn.utils import resample
  from google.cloud import storage
  from num2words import num2words
  from matplotlib.backends.backend_pdf import PdfPages
  import pandas as pd
  import numpy as np
  import random
  import string
  import regex as re
  import matplotlib.pyplot as plt
  import nltk
  import emoji
  nltk.download('stopwords')

except:
  !pip install google-cloud-storage
  !pip install pandas
  !pip install unidecode
  !pip install num2words
  !pip install --upgrade emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, nombre_archivo_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_origen)
    blob = bucket.blob(nombre_blob_origen)
    blob.download_to_filename(nombre_archivo_destino)
    print(f"Archivo {nombre_blob_origen} descargado como {nombre_archivo_destino}.")

def subir_a_bucket_destino(nombre_bucket_procesado, nombre_archivo_destino, nombre_blob_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_procesado)
    blob = bucket.blob(nombre_blob_destino)
    # Verificar si el archivo ya existe en el bucket
    if blob.exists():
        print(f"El archivo {nombre_blob_destino} ya existe en el bucket {nombre_bucket_procesado} y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.")
    else:
        blob.upload_from_filename(nombre_archivo_destino)
        print(f"Archivo {nombre_archivo_destino} subido a {nombre_blob_destino} en el bucket {nombre_bucket_procesado}.")

nombre_bucket_origen = 'ds-edw-processed-d1655b14'
nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_origen = 'CSV/processed/df_concat.csv'
ruta_descarga_local = 'df_concat.csv'
descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, ruta_descarga_local)

input_file_1 = pd.read_csv(ruta_descarga_local)

Archivo CSV/processed/df_concat.csv descargado como df_concat.csv.


**2. Pipeline**

In [11]:
emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # Emoticonos y expresiones faciales
            u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
            u"\U0001F680-\U0001F6FF"  # Transporte y símbolos de mapa
            u"\U0001F3B5-\U0001F3EF"  # Símbolos de nota musical
            u"\U0001F700-\U0001F77F"  # Símbolos alquímicos
            u"\U0001F7E0-\U0001F7EB"  # Símbolos matemáticos
            u"\U0001F0A0-\U0001F0FF"  # Símbolos de cartas de juego
            u"\U0001F000-\U0001F02F"  # Símbolos de mahjong
            u"\U0001F030-\U0001F09F"  # Símbolos de puntuación y flechas
            u"\U0001F200-\U0001F2FF"  # Símbolos de estilo tipográfico
            "]+", flags=re.UNICODE)
def preprocess_data(input_file, output_file, balance_ratio=1):
    # Cargar datos desde el archivo CSV
    data = pd.read_csv(input_file)

    # Eliminar filas con valores NaN
    data.dropna(subset=['text'], inplace=True)

    # Submuestrear la clase mayoritaria para balancear las clases
    df_majority = data[data['label'] == 0]
    df_minority = data[data['label'] == 1]
    df_majority_downsampled = resample(df_majority, replace=False, n_samples=int(len(df_minority) * balance_ratio), random_state=42)
    data_balanced = pd.concat([df_majority_downsampled, df_minority])

    # Guardar el DataFrame procesado en un archivo CSV
    data_balanced.to_csv('df_balanced.csv', index=False)

    # Función para eliminar emojis del texto
    def remove_emojis(text):
        return emoji_pattern.sub('', text)

    # Función para limpiar y procesar el texto
    def clean_text(text):
        # Eliminar emojis
        text = remove_emojis(text)
        # Eliminar nombres de usuario
        text = re.sub(r'@[A-Za-z0-9_]+', '', text)
        # Eliminar puntuación y convertir a minúsculas
        text = re.sub(r'[^\w\s]', '', text)  # Eliminar todos los caracteres de puntuación
        text = unidecode(text)  # Eliminar acentos
        text = text.lower()  # Convertir a minúsculas
        # Eliminar exclamaciones específicamente
        text = text.replace('!', '')
        # Convertir números a palabras
        words = [num2words(word, lang='es') if word.isdigit() else word for word in text.split()]
        text = ' '.join(words)
        return text

    # Aplicar la limpieza al texto
    data_balanced['processed_text'] = data_balanced['text'].apply(clean_text)

     # Eliminar la columna 'text' del DataFrame después de guardar
    data_balanced.drop('text', axis=1, inplace=True)

    # Guardar el DataFrame procesado en un archivo CSV
    data_balanced.to_csv(output_file, index=False)

    return output_file

# Especificar el nombre del archivo de entrada y salida, y el ratio de balanceo
input_file = 'df_concat.csv'
output_file = 'df_processed.csv'
balance_ratio = 1

# Llamar a la función preprocess_data para realizar el preprocesamiento
processed_file = preprocess_data(input_file, output_file, balance_ratio)
print(f'Los datos preprocesados han sido guardados en: {processed_file}')

nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_destino = 'CSV/df_processed.csv'
subir_a_bucket_destino(nombre_bucket_procesado, output_file, nombre_blob_destino)

Los datos preprocesados han sido guardados en: df_processed.csv
El archivo CSV/df_processed.csv ya existe en el bucket ds-edw-processed-d1655b14 y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.


In [12]:
input_file = pd.read_csv(ruta_descarga_local)
df_tab = input_file

def extract_and_count_emojis(text):
    if isinstance(text, str):
        emoji_list = []
        emoji_list = emoji_pattern.findall(text)
        return emoji_list
    else:
        return []  #lista vacía si no es una cadena de texto

# Aplicar la función a la columna 'text' para obtener una lista de emoticonos por fila
df_tab['emoji'] = df_tab['text'].apply(extract_and_count_emojis)

# Crear un DataFrame nuevo df_emoji con todos los emoticonos en una sola columna
all_emoji = [emoji for sublist in df_tab['emoji'] for emoji in sublist]

# Calcular el porcentaje de aparición de cada emoticono con respecto al total de emoticonos
emoji_counts = pd.Series(all_emoji).value_counts()
total_emojis = len(all_emoji)
df_emoji = (emoji_counts / total_emojis * 100).round(2).reset_index()

# Renombrar las columnas en el DataFrame de resultados
df_emoji.columns = ['emoji', '% del total']

# Ordenar el DataFrame por el porcentaje del total de mayor a menor
df_emoji.reset_index(drop=True, inplace=True)

# Guardar el DataFrame ordenado en un archivo CSV
output_file = "df_emoji.csv"
df_emoji.to_csv(output_file, index=False)

nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_destino = 'CSV/processed/df_emoji.csv'
subir_a_bucket_destino(nombre_bucket_procesado, output_file, nombre_blob_destino)

El archivo CSV/processed/df_emoji.csv ya existe en el bucket ds-edw-processed-d1655b14 y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.


**3. Comprobación del procesado**

In [13]:
data_balanced = pd.read_csv('df_balanced.csv')
data_balanced.info()
print("." * 20)
df_processed = pd.read_csv('df_processed.csv')
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22722 entries, 0 to 22721
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    22722 non-null  object
 1   label   22722 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 355.2+ KB
....................
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22722 entries, 0 to 22721
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           22722 non-null  int64 
 1   processed_text  22722 non-null  object
dtypes: int64(1), object(1)
memory usage: 355.2+ KB


In [14]:
# Corpus balanceado sin procesar
print(data_balanced['text'].head(10))

0    ¡Arco Forum recibe este viernes la Medalla de ...
1    las reacciones a la muerte de chacón de todo s...
2    millones de chilenos no votan nunca otros tant...
3    dime a mi y yo te hago reir com osea para que ...
4    se compraron esas aireadoras culiás de papas f...
5    si yo entrego un goya y el que lo gana no está...
6    es cultural de nosotros los ecuatorianos ser m...
7                                             malditos
8    @oscar_mora_ Y educarlos tú, que son tus hijos...
9    y se espera que nuevos flujos de inmigrantes d...
Name: text, dtype: object


In [15]:
# Corpus balanceado procesado
print(df_processed['processed_text'].head(10))

0    arco forum recibe este viernes la medalla de o...
1    las reacciones a la muerte de chacon de todo s...
2    millones de chilenos no votan nunca otros tant...
3    dime a mi y yo te hago reir com osea para que ...
4    se compraron esas aireadoras culias de papas f...
5    si yo entrego un goya y el que lo gana no esta...
6    es cultural de nosotros los ecuatorianos ser m...
7                                             malditos
8         y educarlos tu que son tus hijos te imaginas
9    y se espera que nuevos flujos de inmigrantes d...
Name: processed_text, dtype: object


In [16]:
df_emoji.head(5)

,emoji,% del total
0,🏻,5.99
1,😂,5.34
2,👉,2.77
3,😂😂😂,2.54
4,👇,1.95
